<a href="https://colab.research.google.com/github/onseigousei7/wav_renamer/blob/main/wav_renamer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install espnet_model_zoo

     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 947 kB 29.7 MB/s 
     |████████████████████████████████| 1.2 MB 60.2 MB/s 
     |████████████████████████████████| 212 kB 70.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 71 kB 10.2 MB/s 
     |████████████████████████████████| 86 kB 8.4 MB/s 
     |████████████████████████████████| 596 kB 77.1 MB/s 
     |████████████████████████████████| 1.5 MB 51.2 MB/s 
     |████████████████████████████████| 749 kB 74.3 MB/s 
     |████████████████████████████████| 3.1 MB 80.8 MB/s 
     |████████████████████████████████| 1.3 MB 69.1 MB/s 
     |████████████████████████████████| 235 kB 73.2 MB/s 
     |████████████████████████████████| 180 kB 82.9 MB/s 
  Created wheel for ctc-segmentation: filename=ctc_segmentation-1.7.1-cp37-cp37m-linux_x86_64.whl size=114730 sha256=22471cb4

In [2]:
import soundfile as sf
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text

import librosa
import glob
import pandas as pd
from pathlib import Path
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [3]:
#E2E-ASRモデルのインスタンスを作成します
#音声ファイルからテキストを作成しますが、モーラや音素の生成はできないようです（？）
#https://github.com/espnet/espnet/issues/3456
d = ModelDownloader()
speech2text = Speech2Text(
    #下で使用するモデルだと、音声ファイルの周波数は16kHzである必要があります
    **d.download_and_unpack("kan-bayashi/csj_asr_train_asr_transformer_raw_char_sp_valid.acc.ave")
)

https://zenodo.org/record/4037458/files/asr_train_asr_transformer_raw_char_sp_valid.acc.ave.zip?download=1: 100%|██████████| 412M/412M [00:04<00:00, 96.2MB/s]


In [4]:
def asr(sound_data_path, sample_rate):
  y, sr = librosa.core.load(sound_data_path, sr=sample_rate, mono=True)
  #text, token, *_ = speech2text(y)[0]
  result = speech2text(y)[0]
  return result[0]

def get_path_list(sound_data_folder, extension):
  dir_path = sound_data_folder + '/*.' + extension
  return glob.glob(dir_path)

def generate_asr_csv(csv_path, sound_data_folder, input_sample_rate):
  path_list = get_path_list(sound_data_folder, 'wav')
  text_list = [] 

  for path in path_list:
    print('processing: ', path)
    text = asr(path, input_sample_rate)
    print('text: ', text)
    text_list.append(text)

  dict = {'text': text_list, 'path': path_list} 
  df = pd.DataFrame(dict) 
  df.to_csv(csv_path)

def rename_wav(csv_path):
  df = pd.read_csv(csv_path)

  for i in range(len(df)):
    print('before rename: ', df['path'][i])
    path = Path(df['path'][i])
    rename_path = str(path.parent) + '/' + df['text'][i] + str(path.suffix)
    print('after rename: ', rename_path)
    os.rename(path, rename_path)

In [5]:
#音声データのあるフォルダから、それぞれのパスと書き起こしを記した記したcsvを作成します
#csv_pathとsound_data_folderは各自ご変更ください
csv_path = 'test.csv'
sound_data_folder = '/content/drive/MyDrive/esp_test/wav/'
input_sample_rate = 16000

generate_asr_csv(csv_path, sound_data_folder, input_sample_rate)

processing:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0005.wav


/usr/local/lib/python3.7/dist-packages/espnet2/layers/stft.py:166: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  olens = (ilens - self.n_fft) // self.hop_length + 1
/usr/local/lib/python3.7/dist-packages/espnet/nets/batch_beam_search.py:107: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prev_hyp_ids = top_ids // self.n_vocab


text:  血圧は健康のパラメーターとして重要である
processing:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0002.wav
text:  木曜日経験会段は何の進展もないまま終了しました
processing:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0001.wav
text:  水をマレーシアから買わなくてはならないのです
processing:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0010.wav
text:  巻き試験に備えて本当に気合いを入れて勉強しなきゃ
processing:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0006.wav
text:  週に四回フランスの授業があります
processing:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0004.wav
text:  一週間してそのニュースは本当になった
processing:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0003.wav
text:  上位の議員は私がデーターを歪めたと考発した
processing:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0007.wav
text:  許可書がなければここへは入れない
processing:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0009.wav
text:  無罪の人々は勿論表明された
processing:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0008.wav
text:  大声で泣きながら女の子は母親を探していた


In [6]:
#テキストの通りにリネームする場合はこれを使います
rename_wav(csv_path)

before rename:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0005.wav
after rename:  /content/drive/MyDrive/esp_test/wav/血圧は健康のパラメーターとして重要である.wav
before rename:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0002.wav
after rename:  /content/drive/MyDrive/esp_test/wav/木曜日経験会段は何の進展もないまま終了しました.wav
before rename:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0001.wav
after rename:  /content/drive/MyDrive/esp_test/wav/水をマレーシアから買わなくてはならないのです.wav
before rename:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0010.wav
after rename:  /content/drive/MyDrive/esp_test/wav/巻き試験に備えて本当に気合いを入れて勉強しなきゃ.wav
before rename:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0006.wav
after rename:  /content/drive/MyDrive/esp_test/wav/週に四回フランスの授業があります.wav
before rename:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0004.wav
after rename:  /content/drive/MyDrive/esp_test/wav/一週間してそのニュースは本当になった.wav
before rename:  /content/drive/MyDrive/esp_test/wav/BASIC5000_0003.wav
after rename:  /content/drive/MyDrive/esp_te

In [ ]:
#以下を参照しました
#https://tech.retrieva.jp/entry/2020/12/23/170645
#https://kan-bayashi.github.io/asj-espnet2-tutorial/